## Import Modules

In [100]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from urllib.parse import urlparse, parse_qs

## Set path form chrome drive

In [101]:
path = '/Users/sathwik/Desktop/chromedriver.exe'

In [115]:
browser = webdriver.Chrome()

In [107]:
url = 'https://www.imdb.com/feature/genre/'

In [108]:
browser.get(url)

## Scrap the data

In [109]:
## List of Genre
genre_list = browser.find_elements(By.XPATH, "//a[@class='ipc-chip ipc-chip--on-base-accent2']")
# genre_list

In [110]:
genres = []
for genre in genre_list:
    temp = genre.text
    if temp not in genres:
        genres.append(temp)
# genres

In [111]:

def parse(url):
    # Parse the URL
    parsed_url = urlparse(url)

    # Extract the path and split it
    path_parts = parsed_url.path.split('/')

    # Extract the required movie id (assuming it's the third part in the path)
    id = path_parts[2]

    return id


In [112]:
library = {}
for genre in genres:
    genre_url = f"https://www.imdb.com/search/title/?title_type=feature&genres={genre}"
    browser.get(genre_url)
    movie_name_xpath = "//h3[@class='ipc-title__text']"
    
    movie_list = browser.find_elements(By.XPATH, movie_name_xpath)

    movie_link_xpath = "//a[@class='ipc-title-link-wrapper']"
    movie_link_list = browser.find_elements(By.XPATH, movie_link_xpath)

    movie_ids = []
    for link in movie_link_list:
        temp = parse(link.get_attribute('href'))
        if temp not in movie_ids and len(movie_ids) < 20:
            movie_ids.append(temp)
            
    library[genre] = movie_ids

# print(library)

In [ ]:
reviews = {}
movies_scrapped = {}
for genre in library:
    for movie_id in library[genre]:
        if movie_id in movies_scrapped:
            continue
        movies_scrapped[movie_id] = True   
        movie_url = f"https://www.imdb.com/title/{movie_id}/reviews?sort=submissionDate&dir=desc&ratingFilter=0"
        browser.get(movie_url)

        reviews_list = []

        # Finding and Clicking on spoiler-elements

        buttons = browser.find_elements(By.XPATH, "//div[@class='ipl-expander spoiler-warning'] | //div[@class='expander-icon-wrapper spoiler-warning__control']")

        # Click on the element to trigger the display of the div
        
        for button in buttons:
            button.click()

        browser.implicitly_wait(1) 

        titles = browser.find_elements(By.XPATH, "//a[@class='title']")  
        content_divs = browser.find_elements(By.XPATH, "//div[@class='text'] | //div[@class='text show-more__control'] | //div[@class='text show-more__control clickable']")

        # print(titles)
        # print(content_divs)
        for title, content_div in zip(titles, content_divs):
            heading = title.text.strip()
            description = content_div.get_attribute("innerHTML")
            soup = BeautifulSoup(description, 'html.parser')
            refined_description = soup.get_text(separator=' ')
            reviews_list.append({"title": heading, "content": refined_description})

            if len(reviews_list) == 10:
                break

        movie = browser.find_element(By.XPATH, "//*[@itemprop='name']").text
        reviews[movie] = reviews_list
        
